In [1]:
import pandas as pd
import numpy as np
# from myUtils import pickle_load, symb_perf_stats_vectorized, symb_perf_stats_vectorized_v1
from myUtils import pickle_load, symb_perf_stats_vectorized_v2

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [2]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbols as column names.
  The function first calculates symbols' drawdown, Ulcer-Index, max-drawdown, std(returns),
  std(returns)/Ulcer-Index, CAGR, CAGR/std(returns), CAGR/Ulcer-Index. Then it calculates and returns 
  the group's (i.e. all the symbols) mean/standard-deviation for 'retrun_std/UI', 'CAGR/return_std', 'CAGR/UI'.

  Args:
    df_close(dataframe): dataframe of symbols' close with
      DatetimeIndex e.g. (['2016-12-19', ... '2016-12-22']), symbols as
      column names, and symbols' close as column values.

  Return:
    df_perf(dataframe): dataframe with columns symbol, first date, last date, Year, CAGR,	UI,
    	return_std/UI, CAGR/return_std, CAGR/UI
    grp_retnStd_d_UI_mean_div_std(float): (std(returns)/Ulcer-Index).mean / (std(returns)/Ulcer-Index).std
    grp_CAGR_d_retnStd_mean_div_std(float): (CAGR/std(returns)).mean / (CAGR/std(returns)).std
    grp_CAGR_d_UI_mean_div_std(float): (CAGR/Ulcer_Index).mean / (CAGR/Ulcer_Index).std
  '''

  from myUtils import symb_perf_stats_vectorized_v2

  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, returns_std_div_UI, CAGR, CAGR_div_returns_std, CAGR_div_UI = \
      symb_perf_stats_vectorized_v2(df_close)

  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = df_close.index[0].strftime('%Y-%m-%d')
      date_last = df_close.index[-1].strftime('%Y-%m-%d')

      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], returns_std_div_UI[symbol], CAGR_div_returns_std[symbol], CAGR_div_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)

  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'return_std/UI', 'CAGR/return_std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'retrun_std/UI', 'CAGR/return_std', 'CAGR/UI']
  grp_CAGR_d_UI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGR_d_UI_std  = df_perf['CAGR/UI'].std()
  grp_CAGR_d_UI_mean_div_std = grp_CAGR_d_UI_mean / grp_CAGR_d_UI_std 

  grp_CAGR_d_retnStd_mean = df_perf['CAGR/return_std'].mean()
  grp_CAGR_d_retnStd_std = df_perf['CAGR/return_std'].std()
  grp_CAGR_d_retnStd_mean_div_std = grp_CAGR_d_retnStd_mean / grp_CAGR_d_retnStd_std

  grp_retnStd_d_UI_mean = df_perf['return_std/UI'].mean()
  grp_retnStd_d_UI_std = df_perf['return_std/UI'].std()
  grp_retnStd_d_UI_mean_div_std = grp_retnStd_d_UI_mean / grp_retnStd_d_UI_std

  return df_perf, grp_retnStd_d_UI_mean_div_std, grp_CAGR_d_retnStd_mean_div_std, grp_CAGR_d_UI_mean_div_std

In [3]:
df_perf, grp_retnStd_d_UI_mean_div_std, grp_CAGR_d_retnStd_mean_div_std, grp_CAGR_d_UI_mean_div_std = perf_eval(df_c)
print(f'retnStd/UI   grp_mean/grp_std: {grp_retnStd_d_UI_mean_div_std:>10.6f}')
print(f'CAGR/retnStd grp_mean/grp_std: {grp_CAGR_d_retnStd_mean_div_std:>10.6f}')
print(f'CAGR/UI      grp_mean/grp_std: {grp_CAGR_d_UI_mean_div_std:>10.6f}')
df_perf.head()

retnStd/UI   grp_mean/grp_std:   2.063744
CAGR/retnStd grp_mean/grp_std:  -0.428562
CAGR/UI      grp_mean/grp_std:  -0.021482


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,A,2022-01-05,2023-01-05,1.0,0.030355,0.130170,0.169954,1.372122,0.233197
1,AA,2022-01-05,2023-01-05,1.0,-0.217305,0.416047,0.100341,-5.205345,-0.522309
2,AAL,2022-01-05,2023-01-05,1.0,-0.251071,0.262741,0.135702,-7.041803,-0.955584
3,AAON,2022-01-05,2023-01-05,1.0,-0.064172,0.238438,0.103726,-2.594684,-0.269135
4,AAP,2022-01-05,2023-01-05,1.0,-0.328931,0.229216,0.102647,-13.980296,-1.435029


In [5]:
# %%timeit -n 50 -r 20
my_symbols = ['AE', 'FTSM', 'RMBS', 'WNC']
df_temp = df_c[my_symbols]
df_perf1, grp_retnStd_d_UI_mean_div_std, grp_CAGR_d_retnStd_mean_div_std, grp_CAGR_d_UI_mean_div_std = perf_eval(df_temp)
print(f'retnStd/UI   grp_mean/grp_std: {grp_retnStd_d_UI_mean_div_std:>10.6f}')
print(f'CAGR/retnStd grp_mean/grp_std: {grp_CAGR_d_retnStd_mean_div_std:>10.6f}')
print(f'CAGR/UI      grp_mean/grp_std: {grp_CAGR_d_UI_mean_div_std:>10.6f}')
df_perf1

retnStd/UI   grp_mean/grp_std:   3.047681
CAGR/retnStd grp_mean/grp_std:   0.357004
CAGR/UI      grp_mean/grp_std:   0.565687


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,AE,2022-01-05,2023-01-05,1.0,0.357719,0.136429,0.185573,14.129241,2.622010
1,FTSM,2022-01-05,2023-01-05,1.0,-0.005353,0.005442,0.082681,-11.896441,-0.983614
2,RMBS,2022-01-05,2023-01-05,1.0,0.275170,0.199262,0.128641,10.734904,1.380948
3,WNC,2022-01-05,2023-01-05,1.0,0.104369,0.244056,0.120864,3.538218,0.427643


In [6]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std: nan
grp_CAGRStd_mean_std: nan
grp_CAGRUI_mean_std: nan


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,SPY,2022-01-05,2023-01-05,1.0,-0.190017,0.146743,0.104015,-12.449121,-1.294891
